In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# 2.3 自动求梯度

在深度学习中，我们经常需要对函数求梯度（gradient）。本节将介绍如何使用tensorflow2.0提供的GradientTape来自动求梯度。

## 2.3.1 简单示例

我们先看一个简单例子：对函数 $y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 求关于列向量 $\boldsymbol{x}$ 的梯度。我们先创建变量`x`，并赋初值。

In [2]:
x = tf.reshape(tf.Variable(range(4), dtype=tf.float32), shape=(4,1)) # Variable 可训练参数， constant 不可训练参数
x

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [2.],
       [3.]], dtype=float32)>

函数 $y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 关于$\boldsymbol{x}$ 的梯度应为$4\boldsymbol{x}$。现在我们来验证一下求出来的梯度是正确的。

In [3]:
with tf.GradientTape() as t: # 梯度带，计算梯度，在此 with 下的变量都会被监控来计算梯度的函数和变量
    t.watch(x)
    y = 2 * tf.matmul(tf.transpose(x),x)
y

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[28.]], dtype=float32)>

由于x的形状为（4, 1），y是一个标量。接下来我们可以通过调用 gradient 函数自动求梯度。
函数 $ y=2x^⊤x $ 关于 x  的梯度应为 4x 。现在我们来验证一下求出来的梯度是正确的。
注：tf求梯度函数
```(python)
tf.gradients(
    ys, xs, grad_ys=None, name='gradients', gate_gradients=False,
    aggregation_method=None, stop_gradients=None,
    unconnected_gradients=tf.UnconnectedGradients.NONE
)
```
$y$和$x$都是 `tensor`
更进一步，tf.gradients()接受求导值`ys`和`xs`不仅可以是`tensor`，还可以是`list`，形如[tensor1, tensor2, …, tensorn]。当`ys`和`xs`都是`list`时，它们的求导关系为：
- gradients() adds ops to the graph to output the derivatives of ys with respect to xs. It returns a list of Tensor of length len(xs) where each tensor is the sum(dy/dx) for y in ys.

意思是：
1. tf.gradients()实现ys对xs求导
2. 求导返回值是一个list，list的长度等于len(xs)
3. 假设返回值是[grad1, grad2, grad3]，ys=[y1, y2]，xs=[x1, x2, x3]。则，真实的计算过程为
  - $ grad1 = \frac{dy_1}{dx_1} + \frac{dy_2}{dx_1}$
  - $ grad2 = \frac{dy_1}{dx_2} + \frac{dy_2}{dx_x}$
  - $ grad3 = \frac{dy_1}{dx_3} + \frac{dy_2}{dx_3}$


In [4]:
dy_dx = t.gradient(y,x)
dy_dx

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[ 0.],
       [ 4.],
       [ 8.],
       [12.]], dtype=float32)>

$ x = [0,1,2,3]^T $

$ y = 2x^Tx $

$ y'= 4x = [0,4,8,12]^T $ 

[梯度](https://zh.d2l.ai/chapter_appendix/math.html#%E6%A2%AF%E5%BA%A6)

## stop_gradients 参数

`stop_gradients`也是一个list，list中的元素是tensorflow graph中的op，一旦进入这个list，将不会被计算梯度，更重要的是，在该op之后的BP计算都不会运行。
例如：

```(python)
a = tf.constant(0.)
b = 2 * a
c = a + b
g = tf.gradients(c, [a, b])
```

计算得g = [3.0, 1.0]。因为:

$ b = 2*a $

$ \frac{db}{da} = 2 $

$ \frac{\partial c}{\partial a} = \frac{\partial c}{\partial b}*\frac{db}{da} + \frac{\partial c}{\partial a} = 1*2+1 = 3.0 $

$ \frac{dc}{db} = 1 $

但如果冻结operator a和b的梯度计算：

```(python)
a = tf.constant(0.)
b = 2 * a
g = tf.gradients(a + b, [a, b], stop_gradients=[a, b])
```

$ c = a+b $

$ \frac{dc}{da} = 1.0 $

$ \frac{dc}{db} = 1.0 $

计算得g=[1.0, 1.0]。

其他参数参阅[此处](https://blog.csdn.net/hustqb/article/details/80260002?depth_1-utm_source=distribute.pc_relevant.none-task&utm_source=distribute.pc_relevant.none-task)

### 2.3.2 训练模式和预测模式

GradientTape的资源在调用gradient函数后就被释放，再次调用就无法计算了(申请内存)。所以如果需要多次计算梯度，需要开启persistent=True属性

In [5]:
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x * x
    z = y * y
    dz_dx = g.gradient(z,x) # z = x^4 ; z' = 4x^3 ; x = [0,1,2,3] ; z' = [0,4,32,108]
    dz_dy = g.gradient(z,y) # z = y*y ; z' = 2*y ; y = x*x = [0,1,4,9] ; z' = [0,2,8,18]
dz_dx,dz_dy

(<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[  0.],
        [  4.],
        [ 32.],
        [108.]], dtype=float32)>,
 <tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[ 0.],
        [ 2.],
        [ 8.],
        [18.]], dtype=float32)>)

一般在网络中使用时，不需要显式调用watch函数，使用默认设置，GradientTape会监控可训练变量，例如：

```(python)
with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
gradients = tape.gradient(loss, model.trainable_variables)
```

这样即可计算出所有可训练变量的梯度，然后进行下一步的更新。对于TensorFlow 2.0，推荐大家使用这种方式计算梯度，并且可以在eager模式下查看具体的梯度值。
[参考博客](https://blog.csdn.net/guanxs/article/details/102471843)

### 2.2.3 对Python控制流求梯度

即使函数的计算图包含了Python的控制流（如条件和循环控制），我们也有可能对变量求梯度。(x经过多次乘积运算，仍然可以求梯度)

考虑下面程序，其中包含Python的条件和循环控制。需要强调的是，这里循环（while循环）迭代的次数和条件判断（if语句）的执行都取决于输入a的值。


In [6]:
def f(a):
    b = a * 2
    while tf.norm(b) < 1000: # tf.norm(x,ord='euclidean') 向量矩阵范数
        b = b * 2
        print("{}\n{}\n".format(tf.norm(b),b))
    if tf.reduce_sum(b) > 0: # b中所有元素累加
        c = b
    else:
        c = 100 * b
    return c

我们来分析一下上面定义的`f`函数。事实上，给定任意输入`a`，其输出必然是 `f(a) = x * a` 的形式，其中标量系数`x`的值取决于输入`a`。由于`c = f(a)`有关`a`的梯度为`x`，且值为`c / a`，我们可以像下面这样验证对本例中控制流求梯度的结果的正确性。

In [7]:
a = tf.random.normal(shape=(1,1), dtype=tf.float32)
with tf.GradientTape(persistent=True) as t:
    t.watch(a)
    c = f(a)
a,c

1.95203697681427
[[1.952037]]

3.90407395362854
[[3.904074]]

7.80814790725708
[[7.808148]]

15.61629581451416
[[15.616296]]

31.23259162902832
[[31.232592]]

62.46518325805664
[[62.465183]]

124.93036651611328
[[124.93037]]

249.86073303222656
[[249.86073]]

499.7214660644531
[[499.72147]]

999.4429321289062
[[999.44293]]

1998.8858642578125
[[1998.8859]]



(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.48800924]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1998.8859]], dtype=float32)>)

In [8]:
t.gradient(c,a),c/a,t.gradient(c,a) == c/a

(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4096.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4096.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=bool, numpy=array([[ True]])>)